In [ ]:
import torch
import cv2
import numpy as np
from diffusers.pipelines.stable_video_diffusion.pipeline_stable_video_diffusion import StableVideoDiffusionPipeline
from diffusers.utils.loading_utils import load_image
from diffusers.utils.export_utils import export_to_video
from PIL import Image
import os
from datetime import datetime


In [ ]:

def generate_video(prompt, 
                   output_path="outputs_videos.pm4",
                   model_id="stabilityai/stable-video-diffusion-img2vid-xt",
                   image_path=None,
                   num_frames=14,
                   fps=7,
                   height=320,
                   width=512,
                   motion_bucket_id=127,
                   noise_aug_strength=0.02):
    """
    使用Stable Video Diffusion生成视频
    
    参数:
        prompt: 文本提示词
        output_dir: 输出目录
        model_id: 模型ID
        image_path: 初始图像路径（可选，无则随机生成初始帧）
        num_frames: 生成的帧数
        fps: 帧率
        height: 视频高度
        width: 视频宽度
        motion_bucket_id: 运动强度（0-255，值越大运动越剧烈）
        noise_aug_strength: 噪声增强强度，影响视频变化程度
    """
    # 检查CUDA是否可用
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"使用设备: {device}")
    
    if device != "cuda":
        print("错误: 文生视频模型需要CUDA支持，未检测到GPU加速")
        return None
    
    # 加载模型
    pipe = StableVideoDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        variant="fp16"
    )
    pipe = pipe.to(device)
    
    # 启用xFormers加速（如果安装）
    try:
        pipe.enable_xformers_memory_efficient_attention()
        print("已启用xFormers加速")
    except:
        print("未安装xFormers，使用默认注意力机制")
    
    # 准备初始图像（如果没有提供则生成）
    if image_path:
        init_image = load_image(image_path).resize((width, height))
    else:
        # 如果没有初始图像，先使用文本生成一张初始图像
        from diffusers import StableDiffusionPipeline
        image_pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16
        ).to(device)
        init_image = image_pipe(prompt, width=width, height=height).images[0]
        init_image = init_image.resize((width, height))
    
    # 生成视频帧
    with torch.no_grad():
        frames = pipe(
            init_image,
            prompt=prompt,
            num_frames=num_frames,
            fps=fps,
            motion_bucket_id=motion_bucket_id,
            noise_aug_strength=noise_aug_strength,
            decode_chunk_size=8  # 分块解码节省显存
        ).frames[0]
    
    # 导出为视频
    export_to_video(frames, output_path, fps=fps)
    print(f"视频已保存至: {output_path}")
    
    return output_path


    

In [ ]:
os.makedirs("outputs_videos", exist_ok=True)


prompt = "A beautiful waterfall in a forest, sunlight filtering through trees, smooth flowing water, 4k"

# 生成视频
generate_video(
    prompt=prompt,
    output_path="outputs_videos/waterfall.png",
    num_frames=14,  # RTX 5090移动版建议14-25帧
    fps=7,
    height=320,
    width=512,
    motion_bucket_id=100,  # 中等运动强度
    noise_aug_strength=0.02
)